### N-gram language models or how to write scientific papers

Мы обучим нашу языковую модель на статьх [ArXiv](http://arxiv.org/) и посмотрим, сможем ли мы сгенерировать похожую!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2025-06-29 13:21:15--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... 

connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2025-06-29 13:21:16--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc002842d918e60f9e869b6f305f.dl.dropboxusercontent.com/cd/0/inline/CsjkbFpWR0rHTpTuj6loc0TRuS9RgHE5APk3gbxS4j0CE_vlKQLXX4uAXekzxBXmH1ddNJP1viKgZBMHpGgtIecss3tfsAyEqCa-WExNRpwMiQNMyYTDzXwVqCBr3oyR3yE/file?dl=1# [following]
--2025-06-29 13:21:16--  https://uc002842d918e60f9e869b6f305f.dl.dropboxusercontent.com/cd/0/inline/CsjkbFpWR0rHTpTuj6loc0TRuS9RgHE5APk3gbxS4j0CE_vlKQLXX4uAXekzxBXmH1ddNJP1viKgZBMHpGgtIecss3tfsAyEqCa-WExNRpwMiQNMyYTDzXwVqCBr3oyR3yE/file?dl=1
Resolving uc002842d918e60f9e869b6f305f.dl.dropboxusercontent.com (uc002842d918e60f9e

,author,day,id,link,month,summary,tag,title,year
30177,"[{'name': 'Yu-An Chung'}, {'name': 'Wei-Hung W...",22,1711.08490v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Deep neural networks have been investigated in...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Learning Deep Representations of Medical Image...,2017
3694,"[{'name': 'Ke Li'}, {'name': 'Jitendra Malik'}]",1,1512.00442v3,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Existing methods for retrieving k-nearest neig...,"[{'term': 'cs.DS', 'scheme': 'http://arxiv.org...",Fast k-Nearest Neighbour Search via Dynamic Co...,2015
5974,[{'name': 'Hubert Haoyang Duan'}],3,1402.0459v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,"From a fresh data science perspective, this th...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Applying Supervised Learning Algorithms and a ...,2014
13671,"[{'name': 'A. Ukil'}, {'name': 'J. Bernasconi'...",18,1503.05272v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",3,IR or near-infrared (NIR) spectroscopy is a me...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Improved Calibration of Near-Infrared Spectra ...,2015
24731,"[{'name': 'Julien Mairal'}, {'name': 'Francis ...",12,1411.3230v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,"In recent years, a large amount of multi-disci...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Sparse Modeling for Image and Vision Processing,2014


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Токенизация

Все как обычно, вы уже опытные. Данные кроме тебя никто не токенизирует. Займитесь очисткой данных. Используйте WordPunctTokenizer

In [4]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

for i, text in enumerate(lines):
    text = text.lower()                      # сначала в нижний регистр
    tokens = tokenizer.tokenize(text)          # токенизируем
    lines[i] = ' '.join(tokens)              # записываем обратно
    

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-граммовая языковая модель

Языковая модель - это вероятностная модель, которая оценивает вероятность текста: совместную вероятность всех лексем $w_t$ в тексте $X$: $P(X) = P(w_1, \dots, w_T)$.

Это можно сделать, следуя правилу цепочки:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

Проблема такого подхода заключается в том, что финальный термин $P(w_T \mid w_1, \dots, w_{T-1})$ зависит от $n-1$ предыдущих слов. Эту вероятность нецелесообразно оценивать для длинных текстов, например, $T = 1000$.

Одна из популярных аппроксимаций - предположить, что следующее слово зависит только от конечного количества предыдущих слов:


$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).$$


Такая модель называется __n-gram language model__, где n - параметр. Например, в 3-граммовой языковой модели каждое слово зависит только от двух предыдущих слов.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

Иногда такую аппроксимацию также можно встретить под названием _марковское предположение n-го порядка_.


Первым этапом построения такой модели является подсчет всех вхождений слов с учетом N-1 предыдущих слов

In [8]:
from tqdm import tqdm
from collections import defaultdict, Counter, deque

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    
    for line in tqdm(lines, desc=f"Counting {n}-grams"):
        # разбиваем строку на токены и добавляем EOS
        tokens = line.split() + [EOS]
        # инициализируем префикс UNK-ами
        prefix = deque([UNK] * (n - 1), maxlen=n - 1)
        
        # для каждого токена инкрементируем счётчик
        for tok in tokens:
            counts[tuple(prefix)][tok] += 1
            # сдвигаем префикс: добавляем текущий токен
            if n > 1:
                prefix.append(tok)
    return counts


In [9]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Counting 3-grams: 100%|██████████| 100/100 [00:00<00:00, 35481.80it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [10]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix, next_counts in counts.items():
            total = sum(next_counts.values())                  # общее число всех продолжений
            for token, cnt in next_counts.items():
                self.probs[prefix][token] = cnt / total

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [11]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Counting 3-grams: 100%|██████████| 100/100 [00:00<00:00, 10250.51it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [12]:
lm = NGramLanguageModel(lines, n=3)

Counting 3-grams: 100%|██████████| 41000/41000 [00:06<00:00, 6765.83it/s]


Процесс генерации последовательностей... ну, он последовательный. Вы храните список лексем и итеративно добавляете следующую лексему путем выборки с вероятностями.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Вместо выборки с вероятностями можно также попробовать всегда брать наиболее вероятную лексему, выборку среди топ-K наиболее вероятных лексем или выборку с температурой. В последнем случае (температура) выборка производится из
* $w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$.

Где $\tau > 0$ - температура модели. Если $\tau << 1$, то более вероятные токены будут выбраны с еще большей вероятностью, а менее вероятные исчезнут.

In [15]:
import random
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    # 1) получаем словарь {token: P(token | prefix)}
    probs = lm.get_possible_next_tokens(prefix)
    
    # если нет ни одного продолжения, возвращаем UNK
    if not probs:
        return UNK
    
    tokens, p = zip(*probs.items())  # разделяем токены и вероятности
    
    # 2) temperature == 0 → жадное решение
    if temperature == 0:
        # возвращаем токен с максимальной вероятностью
        return max(probs, key=probs.get)
    
    # 3) корректируем вероятности температурой
    #    new_p[i] = p[i]^(1/temperature)
    adjusted = [prob ** (1.0 / temperature) for prob in p]
    
    # нормируем
    total = sum(adjusted)
    weights = [a / total for a in adjusted]
    
    # 4) семплируем один токен по весам
    return random.choices(tokens, weights)[0]

In [16]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


In [17]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial neural networks have shown the application domains . finally , we use multiple examples to unleash the full 360 - degree polynomial threshold function of discrete dynamical systems and produces sharper and more stable performance in numerous cases of single body shape is perhaps the most widely used to test the hypothesis space . results : on the combination function in cases with a mapreduce implementation of each variable on another . this is illustrated in two diverse datasets without outliers . to develop opponent modeling or solver ) that belong to the availability of huge trees , gaussian graphical


In [18]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the best of our knowledge , this article we study the problem of finding a match among these algorithms are based on the other hand , on the other . this approach is demonstrated by experiments on the other hand , we propose a novel method that is , the proposed approach is demonstrated in a wide range of applications , the proposed method can be used to discover the common assumption that the proposed approach has been shown to be able to generate a high - dimensional space ; ( 2 ) a new approach for the


__Больше в лабе:__ nucleus sampling, top-k sampling, beam search(не для слабонервных).

### Оценка языковых моделей: perplexity (1 балл)

_perplexity - это мера того, насколько хорошо ваша модель аппроксимирует истинное распределение вероятностей, лежащее в основе данных. Меньшая perplexity = лучше модель_.

Чтобы вычислить недоумение на одном предложении, используйте:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


На уровне корпуса текстов perplexity - это произведение вероятностей всех лексем во всех предложениях в степени $1/N$, где $N$ - _общая длина (в лексемах) всех предложений в корпусе текстов_.

Это число может быстро стать слишком маленьким для точности float32/float64, поэтому мы рекомендуем сначала вычислить log-perplexity (из log-probabilities), а затем взять экспоненту.

In [19]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    sum_logprob = 0.0
    total_tokens = 0

    for line in lines:
        tokens = line.split()
        prefix = []  # текущий префикс (список последних n-1 токенов)

        # считаем log-вероятности для каждого токена
        for token in tokens:
            prob = lm.get_next_token_prob(' '.join(prefix), token)
            if prob > 0:
                logp = np.log(prob)
            else:
                logp = min_logprob
            # если logp ещё меньше минимума, то обрезаем
            if logp < min_logprob:
                logp = min_logprob

            sum_logprob += logp
            total_tokens += 1

            # обновляем префикс
            prefix.append(token)
            if len(prefix) > lm.n - 1:
                prefix = prefix[-(lm.n - 1):]

        # учитываем EOS
        prob_eos = lm.get_next_token_prob(' '.join(prefix), EOS)
        if prob_eos > 0:
            logp_eos = np.log(prob_eos)
        else:
            logp_eos = min_logprob
        if logp_eos < min_logprob:
            logp_eos = min_logprob

        sum_logprob += logp_eos
        total_tokens += 1

    # перплексия: exp(−1/N * sum logp)
    return np.exp(- sum_logprob / total_tokens)

In [20]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Counting 10-grams: 100%|██████████| 100/100 [00:00<00:00, 29238.79it/s]

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Давайте теперь измерим perplexity по факту:

In [21]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


Counting 1-grams: 100%|██████████| 30750/30750 [00:01<00:00, 27537.20it/s]


N = 1, Perplexity = 1832.23136


Counting 2-grams: 100%|██████████| 30750/30750 [00:02<00:00, 13503.53it/s]


N = 2, Perplexity = 85653987.28774


Counting 3-grams: 100%|██████████| 30750/30750 [00:04<00:00, 6885.50it/s]


N = 3, Perplexity = 61999196259043346743296.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

Проблема нашей простой языковой модели заключается в том, что всякий раз, когда она встречает n-грамму, которую никогда раньше не видела, она присваивает ей вероятность 0. Каждый раз, когда это происходит, недоумение взрывается.

Для борьбы с этой проблемой существует техника, называемая __сглаживанием__. Суть ее заключается в том, чтобы изменить подсчеты таким образом, чтобы не допустить слишком низкого значения вероятности. Простейшим алгоритмом здесь является аддитивное сглаживание (оно же [сглаживание Лапаса](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

Если количество префиксов невелико, аддитивное сглаживание приведет вероятности к более равномерному распределению. Не стоит забывать, что суммирование в знаменателе идет по _всем словам в словаре_.

In [22]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


In [23]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

Counting 3-grams: 100%|██████████| 100/100 [00:00<00:00, 29044.42it/s]


In [24]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Counting 1-grams: 100%|██████████| 30750/30750 [00:01<00:00, 26867.27it/s]


N = 1, Perplexity = 1832.66878


Counting 2-grams: 100%|██████████| 30750/30750 [00:02<00:00, 13471.98it/s]


N = 2, Perplexity = 470.48021


Counting 3-grams: 100%|██████████| 30750/30750 [00:04<00:00, 7289.45it/s]


N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Сглаживание Кнезера-Нея (4 балла)

Сглаживание Лапласа - простой, достаточно хороший метод, но точно не SOTA


Ваша последняя задача в этом ноутбуке - реализовать сглаживание [Кнезера-Нея](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing).

Оно может быть вычислено рекуррентно, для n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

где
- $prefix_{n-1}$ - кортеж из {n-1} предыдущих лексем
- $lambda_{prefix_{n-1}}$ - константа нормализации, выбранная таким образом, чтобы вероятности складывались в 1
- Униграмма $P_{kn}(w_t | prefix_{n-2})$ соответствует сглаживанию Кнезера-Нея для {N-1}-грамматической языковой модели.
- Униграмма $P_{kn}(w_t)$ - это частный случай: насколько вероятно увидеть x_t в незнакомом контексте.

Более подробные формулы см. на слайдах лекции или в вики.

__Ваша задача__ состоит в том, чтобы
- реализовать класс `KneserNeyLanguageModel`,
- протестировать его на 1-3 грамматических языковых моделях
- найти оптимальную (в пределах разумного) дельту сглаживания для 3-граммовой языковой модели со сглаживанием Кнесер-Нея

In [ ]:
# специальные токены
UNK, EOS = "_UNK_", "_EOS_"

class KneserNeyLanguageModel(NGramLanguageModel):
    """
    Интерполированный Kneser–Ney с абсолютным дисконтом delta.
    """
    def __init__(self, lines, n, delta=1.0):
        assert n >= 1
        self.n = n
        self.delta = delta

        # 1) Собираем счётчики для всех порядков 1..n
        #    count_ngrams(lines, k) возвращает dict(prefix_tuple)->Counter(next_token)
        self.counts = {k: count_ngrams(lines, k) for k in range(1, n+1)}

        # 2) Предвычисляем для каждого порядка:
        #    totals[k][prefix] = sum_c C(prefix→*)
        #    uniques[k][prefix] = |{ w : C(prefix→w)>0 }|
        self.totals  = {}
        self.uniques = {}
        for k in range(1, n+1):
            self.totals[k]  = {pref: sum(cnts.values())
                               for pref, cnts in self.counts[k].items()}
            self.uniques[k] = {pref: len(cnts)
                               for pref, cnts in self.counts[k].items()}

        # 3) continuation_counts: для каждого токена w — в скольких разных
        #    префиксах (order=2) он встречался
        if n >= 2:
            bigrams = self.counts[2]
            cont = Counter()
            for pref, nxt in bigrams.items():
                for w in nxt:
                    cont[w] += 1
            self.continuation_counts = cont
            self.cont_total = sum(cont.values())
        else:
            # для униграмм используем просто raw-частоты
            unigrams = self.counts[1]
            cont = Counter()
            for pref, nxt in unigrams.items():  # pref == ()
                for w, c in nxt.items():
                    cont[w] = c
            self.continuation_counts = cont
            self.cont_total = sum(cont.values())

        # 4) Собираем полный словарь (vocab) всех токенов
        #    Здесь все, что хоть раз встретилось в continuation_counts
        self.vocab = list(self.continuation_counts.keys())

    def _recursive_prob(self, prefix_tuple, token):
        """
        Рекурсивный расчёт P_KN(token | prefix_tuple).
        order = len(prefix_tuple)+1
        """
        order = len(prefix_tuple) + 1

        # ---- база: униграммный Kneser–Ney
        if order == 1:
            return self.continuation_counts.get(token, 0) / self.cont_total

        # основные счётчики
        cnts = self.counts[order].get(prefix_tuple, Counter())
        c_hw = cnts.get(token, 0)
        c_h  = self.totals[order].get(prefix_tuple, 0)

        # back-off префикс = суффикс (остальные n-2 слова)
        backoff_prefix = prefix_tuple[1:] if len(prefix_tuple) > 1 else tuple()
        p_backoff = self._recursive_prob(backoff_prefix, token)

        # нормировочный коэффициент λ(h)
        unique_h = self.uniques[order].get(prefix_tuple, 0)
        λ = (self.delta * unique_h / c_h) if c_h > 0 else 0.0

        # итоговая формула
        p_kn = 0.0
        if c_h > 0:
            p_kn += max(c_hw - self.delta, 0) / c_h
        p_kn += λ * p_backoff
        return p_kn

    def get_next_token_prob(self, prefix, next_token):
        # приводим текстовый префикс к кортежу длины n-1
        toks = prefix.split()
        toks = toks[max(0, len(toks) - (self.n - 1)):]
        toks = [UNK] * ((self.n - 1) - len(toks)) + toks
        prefix_tuple = tuple(toks)
        return self._recursive_prob(prefix_tuple, next_token)

    def get_possible_next_tokens(self, prefix):
        # тот же приём преобразования prefix → tuple
        toks = prefix.split()
        toks = toks[max(0, len(toks) - (self.n - 1)):]
        toks = [UNK] * ((self.n - 1) - len(toks)) + toks
        prefix_tuple = tuple(toks)

        # кандидаты: все, что есть в counts[n][prefix] + весь vocab
        candidates = set(self.counts[self.n].get(prefix_tuple, {}).keys()) \
                   | set(self.vocab)

        result = {}
        for w in candidates:
            p = self.get_next_token_prob(prefix, w)
            if p > 0:
                result[w] = p
        return result

In [29]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

Counting 3-grams: 100%|██████████| 100/100 [00:00<00:00, 57940.38it/s]


In [32]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.75)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Counting 1-grams: 100%|██████████| 30750/30750 [00:01<00:00, 26447.84it/s]


N = 1, Perplexity = 1832.23136


Counting 2-grams: 100%|██████████| 30750/30750 [00:02<00:00, 13323.32it/s]


N = 2, Perplexity = 832.73936


Counting 3-grams: 100%|██████████| 30750/30750 [00:04<00:00, 6471.62it/s]


N = 3, Perplexity = 231976900.22803
